In [ ]:
import sqlite3 as db
import pandas as pd

In [104]:
conn = db.connect("olist.db")
df_tables = pd.read_sql_query (
"""
SELECT name FROM sqlite_master WHERE type = 'table';
"""
,conn)
df_tables

,name
0,olist_order_customer_dataset
1,olist_order_dataset
2,olist_order_reviews_dataset
3,olist_order_payments_dataset
4,olist_order_items_dataset
5,olist_products_dataset
6,olist_sellers_dataset
7,olist_geolocation_dataset
8,product_category_name_translation


In [ ]:
order_customer = pd.read_sql_query("""SELECT * FROM olist_order_customer_dataset;""", conn)
order = pd.read_sql_query("""SELECT * FROM olist_order_dataset;""", conn)
order_reviews = pd.read_sql_query("""SELECT * FROM olist_order_reviews_dataset;""", conn)
order_payments = pd.read_sql_query("""SELECT * FROM olist_order_payments_dataset;""", conn)
order_items = pd.read_sql_query("""SELECT * FROM olist_order_items_dataset;""", conn)
products = pd.read_sql_query("""SELECT * FROM olist_products_dataset;""", conn)
sellers = pd.read_sql_query("""SELECT * FROM olist_sellers_dataset;""", conn)
geolocation = pd.read_sql_query("""SELECT * FROM olist_geolocation_dataset;""", conn)
product_category_name_translation = pd.read_sql_query("""SELECT * FROM product_category_name_translation;""", conn)

In [ ]:
sellers[sellers.duplicated(keep=False)]

In [90]:
null_table = pd.DataFrame()
null_item = [order_customer, order, order_reviews, order_payments, order_items, products, sellers, geolocation]
column_name = ["order_customer", "order", "order_reviews", "order_payments", "order_items","products", "sellers", "geolocation"]
for item in null_item:
    null_table = pd.concat([null_table, item.isnull().sum()], ignore_index=True, axis = 1)
null_table.columns = column_name
null_table.replace(0, None).dropna(how="all")

,order_customer,order,order_reviews,order_payments,order_items,products,sellers,geolocation
order_approved_at,NaN,160.0,NaN,NaN,NaN,NaN,NaN,NaN
order_delivered_carrier_date,NaN,1783.0,NaN,NaN,NaN,NaN,NaN,NaN
order_delivered_customer_date,NaN,2965.0,NaN,NaN,NaN,NaN,NaN,NaN
review_comment_title,NaN,NaN,87656.0,NaN,NaN,NaN,NaN,NaN
review_comment_message,NaN,NaN,58247.0,NaN,NaN,NaN,NaN,NaN
product_category_name,NaN,NaN,NaN,NaN,NaN,610.0,NaN,NaN
product_name_lenght,NaN,NaN,NaN,NaN,NaN,610.0,NaN,NaN
product_description_lenght,NaN,NaN,NaN,NaN,NaN,610.0,NaN,NaN
product_photos_qty,NaN,NaN,NaN,NaN,NaN,610.0,NaN,NaN
product_weight_g,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN


In [98]:
order.dtypes

index                             int64
order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [103]:
order[order["order_delivered_customer_date"].isna()]

,index,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
6,6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,None,None,2017-05-09 00:00:00
44,44,ee64d42b8cf066f35eac1cf57de1aa85,caded193e8e47b8362864762a83db3c5,shipped,2018-06-04 16:44:48,2018-06-05 04:31:18,2018-06-05 14:32:00,None,2018-06-28 00:00:00
103,103,0760a852e4e9d89eb77bf631eaaf1c84,d2a79636084590b7465af8ab374a8cf5,invoiced,2018-08-03 17:44:42,2018-08-07 06:15:14,None,None,2018-08-21 00:00:00
128,128,15bed8e2fec7fdbadb186b57c46c92f2,f3f0e613e0bdb9c7cee75504f0f90679,processing,2017-09-03 14:22:03,2017-09-03 14:30:09,None,None,2017-10-03 00:00:00
154,154,6942b8da583c2f9957e990d028607019,52006a9383bf149a4fb24226b173106f,shipped,2018-01-10 11:33:07,2018-01-11 02:32:30,2018-01-11 19:39:23,None,2018-02-07 00:00:00
...,...,...,...,...,...,...,...,...,...
99283,99283,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,2018-08-31 16:13:44,None,None,None,2018-10-01 00:00:00
99313,99313,e9e64a17afa9653aacf2616d94c005b8,b4cd0522e632e481f8eaf766a2646e86,processing,2018-01-05 23:07:24,2018-01-09 07:18:05,None,None,2018-02-06 00:00:00
99347,99347,a89abace0dcc01eeb267a9660b5ac126,2f0524a7b1b3845a1a57fcf3910c4333,canceled,2018-09-06 18:45:47,None,None,None,2018-09-27 00:00:00
99348,99348,a69ba794cc7deb415c3e15a0a3877e69,726f0894b5becdf952ea537d5266e543,unavailable,2017-08-23 16:28:04,2017-08-28 15:44:47,None,None,2017-09-15 00:00:00


In [111]:
order_canceled = pd.read_sql_query(
    """SELECT order_status, 
    SUM(CASE WHEN order_approved_at is NULL THEN 1 ELSE 0 END) as null_approved,
    SUM(CASE WHEN order_delivered_carrier_date is NULL THEN 1 ELSE 0 END) as null_deliv_carrier,
    SUM(CASE WHEN order_delivered_customer_date is NULL THEN 1 ELSE 0 END) as null_deliv_cust
    FROM olist_order_dataset
    GROUP BY order_status;"""
,conn)
order_canceled

,order_status,null_approved,null_deliv_carrier,null_deliv_cust
0,approved,0,2,2
1,canceled,141,550,619
2,created,5,5,5
3,delivered,14,2,8
4,invoiced,0,314,314
5,processing,0,301,301
6,shipped,0,0,1107
7,unavailable,0,609,609


In [154]:
product_id_check = pd.read_sql_query(
    """SELECT *
    FROM olist_order_items_dataset
    WHERE product_id IN (SELECT product_id FROM olist_products_dataset WHERE product_category_name is NULL)
    ORDER BY shipping_limit_date asc;"""
,conn)
product_id_check

,index,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,4451,0a0837a5eee9e7a9ce2b1fa831944d27,1,583916a5dae918f5e89baec139141c54,3481aa57cd91f9f9d3fa1fa12d9a3bf7,2016-10-10 11:43:22,49.99,15.51
1,79667,b533dea24ada4dc5c1a9041d25d7c812,1,661e3bba6bdb874638367462b49ae590,23613d49c3ac2bd302259e55c06c050c,2016-10-15 04:56:41,15.90,14.52
2,84557,c022b7da8ab7a4a04bd561c8f765a14a,1,d36c8237f8a0a889aa9d6ac7816097e3,80e6699fe29150b372a0c8a1ebf7dcc8,2017-01-21 23:40:42,169.00,25.00
3,43119,6219ece1f053a3ffa649340e87cd93d2,1,aa2b29bee083c3eb3779f39fc09527b7,46dc3b2cc0980fb8ec44634e21d2718e,2017-01-24 15:08:05,349.99,20.69
4,15040,22474d84064cd7eb4a907ba3fd8a6398,1,6f6f3b1a96452e2fbad0cd4f7082765b,ff314fa6033cc68ec451c47aee2d6ba4,2017-01-29 09:09:46,279.90,16.88
...,...,...,...,...,...,...,...,...
1598,68337,9c33774fba00c73b651903f988c71bb9,1,f524a09994004b2dabb3d13cf1a041fb,0c8380b62e38e8a1e6adbeba7eb9688c,2018-08-27 12:30:50,249.90,42.73
1599,101050,e53249d8b27242c64053f94e2af2b2ba,1,b0645512783cc56c9510f08e90a40ede,8648b1e89e9b349e32d3741b30ec737e,2018-08-29 21:35:11,59.90,7.68
1600,44897,6616fa4c89b8bf2a7e17271cdc542fca,2,fc5dd987f12a7b823a76a44aa1ba88f6,49067458c68f7701fd334ce326accbe0,2018-08-30 03:09:38,28.45,6.66
1601,101767,e6f6345c50327507fe35965aa65c8e94,1,fd85f34d70edf41fbea6a3269d183e00,caa9bc43a9fe8cf9c564ddd8a03cc4a6,2018-08-30 19:05:19,29.99,7.47
